<a href="https://colab.research.google.com/github/ctandrewtran/andrewtran/blob/master/asdsaas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install num2words

     |████████████████████████████████| 102kB 2.2MB/s 


In [3]:
import pandas as pd

## WARNING your going to need the python library num2words
from num2words import num2words

#data stored on uconn's github; query parama raw=true needed to retrieve data properly
url = 'https://github.uconn.edu/Ant16112/opim3803/blob/master/GP_data.csv?raw=true'

#reading in csv data
df = pd.read_csv(url)

#making sugar calorie column for later calculation
df['Sugar_Calories'] = df['Sugar'] * 4

#replacing food or drink with integer id to avoid data cleaning
#then replacing integer with string. this allows us to use the for loop down below to automatically generate constraints
#there was an issue with using the food or drink name since there were weird characters like / or %
#there was an issue with using an id based on the index 
#thus num2words was best solution
df['food_drink_id'] = df.index
df['food_drink_id'] = df['food_drink_id'].apply(num2words)
df['food_drink_id']  = df['food_drink_id'].str.replace(" ", '')
df['food_drink_id']  = df['food_drink_id'].str.replace("and", '')
df['food_drink_id']  = df['food_drink_id'].str.replace("-", '')

In [4]:
#showing first 5 rows 
df.head()

,Food or Drink,Calories,Protein,Carb,Sodium,Sugar,Weight Watchers Points,Sugar_Calories,food_drink_id
0,"Original Chili Cheese Dog, standard bun",320,15,31,1410,4,9,16,zero
1,"Original Chili Dog, standard bun",270,12,31,1160,4,8,16,one
2,"Original Mustard Dog, standard bun",240,10,27,810,4,7,16,two
3,"Original Relish Dog, standard bun",250,10,30,870,6,7,24,three
4,"Original Kraut Dog, standard bun",240,10,28,1040,5,7,20,four


# The Diet: IIFYM

Called "if it fits your macros," it is a diet where you simply calculate the calories you burn per day and eat under that threshold alongside protein/fat macro requirements. Any food or drink goes!

## IIFYM Guidlines
Added these in to be more healthy since IIFYM is so flexible

* Calculate basal metabolic rate (BMR) adjusted with total daily energy expenditure (TDEE) and eat 10%-15% below that
* Protein intake is between .7-1 grams per lb of bodyweight

## CDC Health Guidelines
* Sodium must be less than or equal to 2300 mg
* Sugar must bes less than 10% of calories for the day

### Calculate BMR below using the Original Harris-Benedict Equation


In [7]:
#gathering neccesary inputs for basal metabolic rate calculation
print("Are you male or female; input m for male and f for female")
gender = input()

print("Input height in inches")
height = input()
height = float(height)

print("Input age in years")
age = input()
age = float(age)

print("Input weight in lbs")
weight = input()
weight = float(weight)

def bmr_calc(gender, height, age, weight):
  if gender == 'm':
    bmr = 66.4730 + (13.7516 * (weight * .45)) + (5.0033 * (height * 2.54)) - (6.7550 * age)
    return bmr
  elif gender == 'f':
    bmr = 655.0955 + (9.5634 * (weight * .45)) + (1.8496 * (height * 2.54)) - (4.6756  * age)
    return bmr
  else:
    print('Wrong info entered, try again.')

bmr = bmr_calc(gender,height,age,weight)
print("Basal Metabolic Rate is: " + str(bmr))

Are you male or female; input m for male and f for female
m
Input height in inches
65
Input age in years
22
Input weight in lbs
176
Basal Metabolic Rate is: 1833.03455


## BMR Adjustment for activity level

We now adjust BMR by TDEE; essentially adjusting per our exercise level

Encode by inputting Sedentary, lightly active, etc as outlined below

*   Sedentary (desk job, little excercise)
*   Lightly Active (light excercise/sports 1-3 days a week)
*   Moderately Active (moderate excercise/sports 6-7 days a week)
*   Very Active (hard excercise every day, or 2x a day excercise)
*   Extra Active (hard exercise 2+ times a day, or marathon training)


In [8]:
print("How active are you? Input activity rate.")
activity_rate = input()

if activity_rate == 'Sedentary':
  bmr = bmr * 1.2
  print("Adjusted BMR is now: " + str(bmr))
elif activity_rate == 'Lightly Active':
  bmr = bmr * 1.375
  print("Adjusted BMR is now: " + str(bmr))
elif activity_rate == 'Moderately Active':
  bmr = bmr * 1.55
  print("Adjusted BMR is now: " + str(bmr))
elif activity_rate == 'Very Active':
  bmr = bmr * 1.725
  print("Adjusted BMR is now: " + str(bmr))
elif activity_rate == 'Extra Active':
  bmr = bmr * 1.9
  print("Adjusted BMR is now: " + str(bmr))
else:
  print('Wrong input. Try again.')

How active are you? Input activity rate.
Lightly Active
Adjusted BMR is now: 2520.42250625


## Other Constraint calculations done below

In [9]:
#calculating out the bounds 

#protein has to be this many grams per lb of body weight
protein_lb = weight * .7
protein_ub = weight * 1

#sugar has to be less than 10% of your daily calories
#sodium has to be less than 2300 mg
sugar_cal_b = bmr * .1
sodium_b = 2300

#adjusting bmr for weightloss
adj_bmr = bmr * .875

print("Protein lower and upper bound are: " + str(protein_lb) + ", " +  str(protein_ub))
print("Sugar calorie  bound is: " + str(sugar_cal_b))
print("Sodium lower bound is: " + str(sodium_b))
print("Adjusted BMR for Weightloss is: " + str(adj_bmr))

Protein lower and upper bound are: 123.19999999999999, 176.0
Fat lower and upper bound are: 44.0, 70.4
Sugar calorie  bound is: 252.042250625
Sodium lower bound is: 2300
Adjusted BMR for Weightloss is: 2205.36969296875


#**Everything Below is Testing**

Was able to create vars for all items. not sure about constraints

In [ ]:
df.head()

,Food or Drink,Calories,Protein,Carb,Sodium,Sugar,Weight Watchers Points,Sugar_Calories,food_drink_id
0,"Original Chili Cheese Dog, standard bun",320,15,31,1410,4,9,16,zero
1,"Original Chili Dog, standard bun",270,12,31,1160,4,8,16,one
2,"Original Mustard Dog, standard bun",240,10,27,810,4,7,16,two
3,"Original Relish Dog, standard bun",250,10,30,870,6,7,24,three
4,"Original Kraut Dog, standard bun",240,10,28,1040,5,7,20,four


In [37]:
from pyomo.environ import *
SOLVER = 'glpk'
EXECUTABLE = '/usr/bin/glpsol'

#initialize model
model = ConcreteModel()

#loop through and create variables
model.product = Var(df['food_drink_id'].tolist(),domain=NonNegativeReals)

#calorie constraint
model.cal_const = ConstraintList()
cal_const = ""
for x in df.values:
  cal_const += x[1] * model.product[x[8]]
model.cal_const.add(expr = cal_const >= adj_bmr)

#protein constraint
model.prot_const = ConstraintList()
prot_const = ""
for x in df.values:
  prot_const += x[2] * model.product[x[8]]
model.prot_const.add(expr = prot_const >= protein_lb)
model.prot_const.add(expr = prot_const <= protein_ub)

#sugar constraint
model.sugar_const = ConstraintList()
sugar_const = ""
for x in df.values:
  sugar_const += x[7] * model.product[x[8]]
model.sugar_const.add(expr = sugar_const <= sugar_cal_b)

#sodium constraint
model.sodium_const = ConstraintList()
sodium_const = ""
for x in df.values:
  sodium_const += x[4] * model.product[x[8]]
model.sodium_const.add(expr = sodium_const <= sodium_b)

#objective function
obj_expr = ''
for x in df.values:
  obj_expr += x[6] * model.product[x[8]]
model.weight_watchers = Objective(expr = obj_expr, sense=minimize)

In [38]:
# solve it
SolverFactory(SOLVER, executable=EXECUTABLE).solve(model).write()

# show the results
print("Cost = ", model.weight_watchers())

for x in df:
  print(product_name[product], model.product[x[8]]())

AttributeError: ignored

In [36]:

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

from pyomo.environ import *

SOLVER = 'glpk'
EXECUTABLE = '/usr/bin/glpsol'